# Generate Regex JSON from the Alphabet and Punctuation JSON

In [11]:
from unicodedata import normalize
from pathlib import Path
import json
import collections

nena_dir = Path.home().joinpath('github/CambridgeSemiticsLab/nena_corpus')
standards = nena_dir.joinpath('standards')
alpha_file = standards.joinpath('alphabet/alphabet.json')
punct_file = standards.joinpath('punctuation/punctuation.json')
alpha_data = json.loads(alpha_file.read_text())
punct_data = json.loads(punct_file.read_text())
regex_file = standards.joinpath('NFD_regexes.json')

In [12]:
alpha_patts = []
for letter in alpha_data:
    alpha_patts.append(letter['decomposed_regex'])
    
punct_patts = []
for punct in punct_data:
    punct_patts.append(punct['decomposed_regex'])
                             
re_dict = {
    'alphabet': '|'.join(alpha_patts),
    'punctuation': '|'.join(punct_patts),
}

In [13]:
re_dict['alphabet']

'b(?![̀-ͯ])|B(?![̀-ͯ])|c(?![̀-ͯ])|C(?![̀-ͯ])|č(?![̀-ͯ])|Č(?![̀-ͯ])|c̭(?![̀-ͯ])|C̭(?![̀-ͯ])|č̣(?![̀-ͯ])|Č̣(?![̀-ͯ])|č̭(?![̀-ͯ])|Č̭(?![̀-ͯ])|d(?![̀-ͯ])|D(?![̀-ͯ])|ḍ(?![̀-ͯ])|Ḍ(?![̀-ͯ])|f(?![̀-ͯ])|F(?![̀-ͯ])|g(?![̀-ͯ])|G(?![̀-ͯ])|ġ(?![̀-ͯ])|Ġ(?![̀-ͯ])|h(?![̀-ͯ])|H(?![̀-ͯ])|ḥ(?![̀-ͯ])|Ḥ(?![̀-ͯ])|j(?![̀-ͯ])|J(?![̀-ͯ])|k(?![̀-ͯ])|K(?![̀-ͯ])|k̭(?![̀-ͯ])|K̭(?![̀-ͯ])|l(?![̀-ͯ])|L(?![̀-ͯ])|ḷ(?![̀-ͯ])|Ḷ(?![̀-ͯ])|m(?![̀-ͯ])|M(?![̀-ͯ])|ṃ(?![̀-ͯ])|Ṃ(?![̀-ͯ])|n(?![̀-ͯ])|N(?![̀-ͯ])|p(?![̀-ͯ])|P(?![̀-ͯ])|p̣(?![̀-ͯ])|P̣(?![̀-ͯ])|p̭(?![̀-ͯ])|P̭(?![̀-ͯ])|q(?![̀-ͯ])|Q(?![̀-ͯ])|r(?![̀-ͯ])|R(?![̀-ͯ])|ṛ(?![̀-ͯ])|Ṛ(?![̀-ͯ])|s(?![̀-ͯ])|S(?![̀-ͯ])|š(?![̀-ͯ])|Š(?![̀-ͯ])|ṣ(?![̀-ͯ])|Ṣ(?![̀-ͯ])|t(?![̀-ͯ])|T(?![̀-ͯ])|ṭ(?![̀-ͯ])|Ṭ(?![̀-ͯ])|ṱ(?![̀-ͯ])|Ṱ(?![̀-ͯ])|v(?![̀-ͯ])|V(?![̀-ͯ])|w(?![̀-ͯ])|W(?![̀-ͯ])|x(?![̀-ͯ])|X(?![̀-ͯ])|y(?![̀-ͯ])|Y(?![̀-ͯ])|z(?![̀-ͯ])|Z(?![̀-ͯ])|ž(?![̀-ͯ])|Ž(?![̀-ͯ])|ẓ(?![̀-ͯ])|Ẓ(?![̀-ͯ])|ð(?![̀-ͯ])|Ð(?![̀-ͯ])|ð̣(?![̀-ͯ])|Ð̣(?![̀-ͯ])|ɟ(?![̀-ͯ])|ɟ(?![̀-ͯ])|ʾ(?![̀-ͯ])|ʾ(?![̀-ͯ])|ʿ(?![̀-ͯ])|ʿ(?![̀

In [14]:
re_dict['punctuation']

'⁺|(?<= )"| |-|=|ˈ|,|\\.\\.\\.|:|—|;|(?<!\\.)\\.(?!\\.)|\\?|!|(?<! )"'

In [18]:
with open(regex_file, 'w') as outfile:
    json.dump(re_dict, outfile, ensure_ascii=False, indent=4)